In [3]:
pip install sumy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 25.6 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21693 sha256=e8ec38dcceee435a050cd2ce09562a8a4001571880cc953656dfcecbebf1c329
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=c735540d7684cf55e36e652d035dd0cb71da4d6928bea58e852df0a4d5e29ab2
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt


In [4]:
import nltk
import random
from nltk.corpus import stopwords
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.luhn import LuhnSummarizer

In [5]:
# NLTK downloads
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
# Predefined responses for greetings
GREETING_KEYWORDS = ["hello", "hi", "hey", "greetings", "what's up"]
GREETING_RESPONSES = ["Hello!", "Hi there!", "Hey!", "Greetings!", "Hi!"]

HEALTH_INFO = {
    "fever": {
        "symptoms": "Symptoms: High body temperature, sweating, shivering, headache, muscle aches, loss of appetite, dehydration.",
        "first_aid": "First Aid: Rest, drink plenty of fluids, take over-the-counter fever reducers (e.g., acetaminophen, ibuprofen).",
        "healthy_tips": "Healthy Tips: Stay hydrated, rest, avoid alcohol and caffeine, eat light and easily digestible foods.",
        "exercise_suggestion": "Exercise Suggestion: Rest is important during fever. Gentle stretching exercises can be done if comfortable."
    },
    "headache": {
        "symptoms": "Symptoms: Pain or discomfort in the head, scalp, or neck area.",
        "first_aid": "First Aid: Rest in a quiet, dark room, apply cold or warm compresses to the head, take over-the-counter pain relievers.",
        "healthy_tips": "Healthy Tips: Stay hydrated, practice good posture, manage stress, get regular sleep, avoid triggers (e.g., certain foods, loud noise).",
        "exercise_suggestion": "Exercise Suggestion: Gentle neck and shoulder stretches, yoga, or low-impact cardio exercises may help relieve tension headaches."
    },
    "stress": {
        "symptoms": "Symptoms: Feeling overwhelmed, anxious, irritable, difficulty concentrating, muscle tension, headaches, fatigue.",
        "first_aid": "First Aid: Practice relaxation techniques (deep breathing, meditation, yoga), engage in hobbies, talk to a friend or therapist.",
        "healthy_tips": "Healthy Tips: Practice stress management techniques regularly, prioritize tasks, maintain a healthy work-life balance, get regular exercise and sleep.",
        "exercise_suggestion": "Exercise Suggestion: Activities like yoga, tai chi, or walking can help reduce stress and promote relaxation."
    },
    "back pain": {
        "symptoms": "Symptoms: Pain, stiffness, or discomfort in the back area, may radiate to other parts of the body.",
        "first_aid": "First Aid: Rest, apply ice or heat packs to the affected area, take over-the-counter pain relievers, use supportive pillows or cushions.",
        "healthy_tips": "Healthy Tips: Maintain good posture, use ergonomic furniture, lift heavy objects properly, avoid prolonged sitting or standing.",
        "exercise_suggestion": "Exercise Suggestion: Low-impact exercises like swimming, walking, or gentle stretching can help relieve back pain."
    },
    "insomnia": {
        "symptoms": "Symptoms: Difficulty falling asleep, staying asleep, waking up too early, feeling tired upon waking, irritability, difficulty concentrating.",
        "first_aid": "First Aid: Establish a bedtime routine, create a comfortable sleep environment, avoid caffeine and electronics before bedtime, limit daytime naps.",
        "healthy_tips": "Healthy Tips: Stick to a regular sleep schedule, limit screen time before bed, avoid heavy meals and caffeine close to bedtime.",
        "exercise_suggestion": "Exercise Suggestion: Regular aerobic exercise during the day can improve sleep quality. Avoid vigorous exercise close to bedtime."
    },
}

In [13]:
# Function to preprocess text
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    words = nltk.word_tokenize(text.lower())
    filtered_words = [word for word in words if word.isalnum() and word not in stop_words]
    return " ".join(filtered_words)

In [14]:
# Function to generate response using text summarization
def generate_response(user_input):
    # Preprocess user input
    user_input = preprocess_text(user_input)
    # Check if user input matches any health topic
    for health_topic in HEALTH_INFO.keys():
        if health_topic in user_input:
            health_data = HEALTH_INFO[health_topic]
            # Summarize health information
            symptoms = health_data['symptoms']
            parser = PlaintextParser.from_string(symptoms, Tokenizer("english"))
            summarizer = LsaSummarizer()
            summary = summarizer(parser.document, sentences_count=1)  # Summarize to 1 sentence
            summary_text = " ".join([str(sentence) for sentence in summary])
            # Construct response including summarized symptoms, first aid, healthy tips, and exercise suggestions
            response = f"{summary_text}\n{health_data.get('first_aid', '')}\n{health_data.get('healthy_tips', '')}\n{health_data.get('exercise_suggestion', '')}"
            return response
    # If no health topic is recognized, provide a generic response
    return "I'm sorry, I don't have information about that health topic."



In [15]:
# Function to handle greetings
def handle_greetings(user_input):
    for word in user_input.split():
        if word.lower() in GREETING_KEYWORDS:
            return random.choice(GREETING_RESPONSES)
    return None


In [16]:
# Chatbot loop
print("HealthAssist: Hello! I'm HealthAssist, your health and wellness chatbot.")
print("HealthAssist: I can provide information and guidance on various health topics.")
print("HealthAssist: Feel free to ask me anything!")
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("HealthAssist: Goodbye! Take care!")
        break
    greeting_response = handle_greetings(user_input)
    if greeting_response:
        print("HealthAssist:", greeting_response)
    else:
        response = generate_response(user_input)
        print("HealthAssist:",response)

HealthAssist: Hello! I'm HealthAssist, your health and wellness chatbot.
HealthAssist: I can provide information and guidance on various health topics.
HealthAssist: Feel free to ask me anything!
You: hey
HealthAssist: Greetings!
You: fever
HealthAssist: Symptoms: High body temperature, sweating, shivering, headache, muscle aches, loss of appetite, dehydration.
First Aid: Rest, drink plenty of fluids, take over-the-counter fever reducers (e.g., acetaminophen, ibuprofen).
Healthy Tips: Stay hydrated, rest, avoid alcohol and caffeine, eat light and easily digestible foods.
Exercise Suggestion: Rest is important during fever. Gentle stretching exercises can be done if comfortable.
You: exit
HealthAssist: Goodbye! Take care!
